In [0]:
# Costants
BEARER_TOKEN = "INSERT_BEARER_TOKEN"

In [0]:
#Restart this cell when the cluster changes
!pip install --upgrade pip
!pip install tweepy==4.12.1

####Twitter connection

In [0]:
import tweepy
import socket
import json

In [0]:
# Delete previous query

twitter_stream = tweepy.StreamingClient(BEARER_TOKEN)

rule_ids = []
result = twitter_stream.get_rules()
try:
    for rule in result.data:
        print(f"rule marked to delete: {rule.id} - {rule.value}")
        rule_ids.append(rule.id)
    if (len(rule_ids) > 0):
        twitter_stream.delete_rules(rule_ids)
        twitter_stream = tweepy.StreamingClient(BEARER_TOKEN)
    else:
        print("no rules to delete")
except BaseException as e:
    print(f"Error on_data: {e}")

In [0]:
class TweetsListener(tweepy.StreamingClient):
    
    def __init__(self, *args, csocket):
        super().__init__(*args)
        self.client_socket = csocket
    
    def on_connect(self):
        print("Connected!")
    
    def on_data(self, data):
        
        all_data = json.loads(data)
        
        print("-"*50)
        print(f"Il json 'data' contiene: \n {all_data} \n")
        
        tweet = all_data['data']['text']

        print(f"Il testo del tweet è: \n {tweet} \n")
            
        # add at the end of each tweet "t_end"
        try:
            self.client_socket\
            .send(str(tweet + "t_end")\
            .encode('utf-8'))
            
            print("Testo inviato")
            return True

        except BaseException as e:
            print(f"Error on_data: {e}")
            return True
        
    def on_error(self, status):
        
        print(status)
        return True

####Send data from Twitter

In [0]:
def send_data(c_socket):
    print('Start sending data from Twitter to socket')
    twitter_stream = TweetsListener(BEARER_TOKEN, csocket=c_socket)
    twitter_stream.add_rules(tweepy.StreamRule('("Dwayne Johnson" OR @TheRock) -is:retweet lang:en')) #'("Elon Musk" OR @elonmusk) -is:retweet lang:en'
    twitter_stream.filter()
    

In [0]:
# server (local machine) creates listening socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = "localhost"
port = 3412
s.bind((host, port))
print('Socket is ready')
# server (local machine) listens for connections
s.listen(1)
print('Socket is listening')
# return the socket and the address on the other side of the connection (client side)
c_socket, addr = s.accept()
print(f"Received request from: {addr}")
send_data(c_socket)
    